# Evolutionary Computation for Evolving Neural Network EMG Classifier Structure and Weights
In this project, the goal is to improve the structure and weights of a neural network that classifies EMG signals, and compare the genetic solutions to standard classifiers.

Controls:
- Random Forest Classifier
- Neural Network (Backpropagation)

Evolutionary Models:
- NEAT (NeuroEvolution of Augmenting Topologies)
- HyperNEAT

In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
# Imports
from __future__ import annotations
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# Custom library
from emg_evo_lib_kb import *

In [ ]:
# Dataset root
root_path = Path("/Users/kaanborekci/Desktop/CS496/Code/Self-Project-Evolutionary-Computation/EMG-Project/UC-Irvine-Set/EMG_data_for_gestures-master")

# Column names in each raw file
emg_column_names = ["Time"] + [f"ch{i}" for i in range(1, 9)] + ["Class"]

# Gesture classes we care about (ignore 0 = unmarked)
gesture_classes = [1, 2, 3, 4, 5, 6, 7]

In [34]:
files = iter_emg_files(root_path, pattern="*.txt")
files[0]

PosixPath('/Users/kaanborekci/Desktop/CS496/Code/Self-Project-Evolutionary-Computation/EMG-Project/UC-Irvine-Set/EMG_data_for_gestures-master/01/1_raw_data_13-12_22.03.16.txt')

In [ ]:
df = load_emg_file(files[2], emg_column_names)

In [36]:
df

,Time,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,Class
0,1,-0.00001,-0.00003,-0.00002,0.00000,-0.00002,-0.00001,0.00000,-0.00002,0
1,4,-0.00001,-0.00003,-0.00002,0.00000,-0.00002,-0.00001,0.00000,-0.00002,0
2,5,-0.00001,-0.00003,-0.00002,0.00000,-0.00002,-0.00001,0.00000,-0.00002,0
3,6,-0.00001,-0.00003,-0.00002,0.00000,-0.00002,-0.00001,0.00000,-0.00002,0
4,7,-0.00001,-0.00003,-0.00002,0.00000,-0.00002,-0.00001,0.00000,-0.00002,0
...,...,...,...,...,...,...,...,...,...,...
72317,74521,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0
72318,74522,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0
72319,74523,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0
72320,74524,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0


In [42]:
segments = segment_gestures(df, gesture_classes)

NameError: name 'segment_gestures' is not defined